In [1]:
# Import objects from pyomo package
from pyomo.environ import (ConcreteModel,
                           SolverFactory,
                           Constraint,
                           Expression,
                           TransformationFactory,
                           value,
                           units as pyunits)

# Import the main FlowsheetBlock from IDAES. The flowsheet block will contain the unit model
from idaes.core import FlowsheetBlock

# Import idaes logger to set output levels
import idaes.logger as idaeslog

from idaes.generic_models.properties.core.generic.generic_property import (
        GenericParameterBlock)

# Todo: import Flash unit model from idaes.generic_models.unit_models
from idaes.generic_models.unit_models import Flash

In [2]:
from idaes.core.util.model_statistics import degrees_of_freedom, large_residuals_set

import pyomo.contrib.parmest.parmest as parmest

In [3]:
# Import plotting functions
import matplotlib.pyplot as plt

# Import numpy library 
import numpy as np

# Import Pandas
import pandas as pd

In [4]:
from HFCS_bmimPF6_PR import configuration

# Load data from csv
data = pd.read_csv('HFCSIL.csv')

In [5]:
def PR_model(data):
    
    m = ConcreteModel()

    m.fs = FlowsheetBlock(default={"dynamic": False})

    m.fs.properties = GenericParameterBlock(default=configuration)

    m.fs.state_block = m.fs.properties.state_block_class(
        default={"parameters": m.fs.properties,
                 "defined_state": True})

    m.fs.state_block.flow_mol.fix(1)
    m.fs.state_block.temperature.fix(298.15)
    m.fs.state_block.pressure.fix(500000)
    m.fs.state_block.mole_frac_comp["R32"].fix(0.7)
    m.fs.state_block.mole_frac_comp["R125"].fix(0.05)
    m.fs.state_block.mole_frac_comp["bmimPF6"].fix(0.7)
    
    m.fs.properties.PR_kappa['R32', 'R125'].fix(-0.002535)
    m.fs.properties.PR_kappa['R125', 'R32'].fix(-0.002535)
    m.fs.properties.PR_kappa['R32', 'bmimPF6'].fix(-0.03)
    m.fs.properties.PR_kappa['bmimPF6', 'R32'].fix(-0.068)
    m.fs.properties.PR_kappa['bmimPF6', 'R125'].fix(0.086)
    m.fs.properties.PR_kappa['R125', 'bmimPF6'].fix(0.02)
    
    # Initialize the flash unit
    m.fs.state_block.initialize(outlvl=idaeslog.CRITICAL)

    # Fix at actual temperature
    m.fs.state_block.temperature.fix(float(data["temperature"]))
#     m.fs.state_block.pressure.fix(float(data["pressure"]))
    m.fs.state_block.pressure.unfix()
    m.fs.state_block.mole_frac_comp["R125"].unfix()
    m.fs.state_block.mole_frac_comp["R32"].unfix()
    m.fs.state_block.mole_frac_comp["bmimPF6"].unfix()
#     m.fs.state_block.mole_frac_phase_comp['Vap', 'R32'].fix(float(data["y_R32"]))
#     m.fs.state_block.mole_frac_phase_comp['Vap', 'R125'].fix(float(data["y_R125"]))   
    m.fs.state_block.mole_frac_phase_comp['Liq', 'R32'].fix(float(data["x_R32"]))
    m.fs.state_block.mole_frac_phase_comp['Liq', 'R125'].fix(float(data["x_R125"])) 
    m.fs.state_block.mole_frac_phase_comp['Liq', 'bmimPF6'].fix(float(data["x_IL"])) 
    
    # Set bounds on variables to be estimated
    m.fs.properties.PR_kappa['R32', 'R125'].setlb(-3)
    m.fs.properties.PR_kappa['R32', 'R125'].setub(3)

    m.fs.properties.PR_kappa['R125', 'R32'].setlb(-3)
    m.fs.properties.PR_kappa['R125', 'R32'].setub(3)

    m.fs.properties.PR_kappa['R32', 'bmimPF6'].setlb(-3)
    m.fs.properties.PR_kappa['R32', 'bmimPF6'].setub(3)

    m.fs.properties.PR_kappa['bmimPF6', 'R32'].setlb(-3)
    m.fs.properties.PR_kappa['bmimPF6', 'R32'].setub(3)
    
    m.fs.properties.PR_kappa['bmimPF6', 'R125'].setlb(-3)
    m.fs.properties.PR_kappa['bmimPF6', 'R125'].setub(3)

    m.fs.properties.PR_kappa['R125', 'bmimPF6'].setlb(-3)
    m.fs.properties.PR_kappa['R125', 'bmimPF6'].setub(3)
    
    # Return initialized flash model
    return m

In [6]:
import pytest

test_data = {"temperature": 298.15, "pressure": 100000, "x_R32":0.7312, "x_IL":0.7312,  
             "x_R125":0.2688, "y_R125":0.2501, "y_R32":0.7499, "y_IL":0.7312}

m = PR_model(test_data)

DOF_initial = degrees_of_freedom(m)
print("The initial DOF is {0}".format(DOF_initial))

2021-03-14 14:55:20 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
The initial DOF is 1


In [7]:
variable_name = [
#                 "fs.properties.PR_kappa['R32', 'R125']",
#                  "fs.properties.PR_kappa['R125', 'R32']",
                "fs.properties.PR_kappa['R32', 'bmimPF6']",
                 "fs.properties.PR_kappa['bmimPF6', 'R32']",
                 "fs.properties.PR_kappa['bmimPF6', 'R125']",
                 "fs.properties.PR_kappa['R125', 'bmimPF6']"]

In [8]:
def SSE(m, data):
    expr = ((float(data["pressure"]) - m.fs.state_block.pressure)**2)
    return expr*1e-4

# def SSE(m, data):
#     expr = (
#             (float(data["y_R125"]) -
#              m.fs.state_block.mole_frac_phase_comp["Vap", "R125"])**2
# #             + (float(data["x_R125"]) -
# #              m.fs.state_block.mole_frac_phase_comp["Liq", "R125"])**2
#            )

In [9]:
solver_options = {'tol': 1e-4}
pest = parmest.Estimator(PR_model, data, variable_name, SSE, solver_options)
# solver_options = {'tol': 1e-6, 'max_iter':100}

obj_value, parameters = pest.theta_est()

2021-03-14 14:55:20 [WARNING] idaes.generic_models.properties.core.generic.generic_property: fs.properties phase Liq was not assigned a type. Using generic Phase object.
2021-03-14 14:55:20 [WARNING] idaes.generic_models.properties.core.generic.generic_property: fs.properties phase Vap was not assigned a type. Using generic Phase object.
2021-03-14 14:55:20 [WARNING] idaes.generic_models.properties.core.generic.generic_property: fs.properties component bmimPF6 was not assigned a type. Using generic Component object.
2021-03-14 14:55:20 [WARNING] idaes.generic_models.properties.core.generic.generic_property: fs.properties component R32 was not assigned a type. Using generic Component object.
2021-03-14 14:55:20 [WARNING] idaes.generic_models.properties.core.generic.generic_property: fs.properties component R125 was not assigned a type. Using generic Component object.
2021-03-14 14:55:23 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.


2021-03-14 14:55:42 [WARNING] idaes.generic_models.properties.core.generic.generic_property: fs.properties component bmimPF6 was not assigned a type. Using generic Component object.
2021-03-14 14:55:42 [WARNING] idaes.generic_models.properties.core.generic.generic_property: fs.properties component R32 was not assigned a type. Using generic Component object.
2021-03-14 14:55:42 [WARNING] idaes.generic_models.properties.core.generic.generic_property: fs.properties component R125 was not assigned a type. Using generic Component object.
2021-03-14 14:55:45 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-03-14 14:55:45 [WARNING] idaes.generic_models.properties.core.generic.generic_property: fs.properties phase Liq was not assigned a type. Using generic Phase object.
2021-03-14 14:55:45 [WARNING] idaes.generic_models.properties.core.generic.generic_property: fs.properties phase Vap was not assigned a type. Using generic Phase object.


Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
  27  5.5252635e+06 2.26e+02 4.44e+12  -1.0 5.67e+05   0.0 5.66e-02 1.61e-04f 10
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error in an AMPL evaluation. Run with "halt_on_amp

Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error in an AMPL evaluation. Run with "halt_on_ampl_

Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
  35  5.5251963e+06 2.26e+02 1.27e+13  -1.0 5.67e+05  -0.2 8.99e-03 9.42e-09f 22
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error in an AMPL evaluation. Run with "halt_on_amp

...

Scaling factors are invalid - setting them all to 1.
Scaling factors are invalid - setting them all to 1.
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
 128  1.6310367e+05 2.80e+01 2.57e+09  -1.0 3.70e+03   0.2 8.19e-01 1.24e-03f  7
Scaling factors are invalid - setting them all to 1.
Scaling factors are invalid - setting them all to 1.
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see detai

Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
 148  1.5103280e+05 1.08e+01 4.19e+05  -1.0 9.56e+04    -  4.16e-03 7.13e-04f  6
Scaling factors are invalid - setting them all to 1.
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
 149  1.5082283e+05 1.00e+01 2.60e+04  -1.0 9.19e+04    -  6.01e-02 7.39e-04f  6
Scaling factors are invalid - setting them all to 1.
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error in an AMPL eval

Scaling factors are invalid - setting them all to 1.
Scaling factors are invalid - setting them all to 1.
 167  5.9082644e+04 3.96e+01 3.14e+10  -1.0 3.33e+03   1.9 7.61e-01 6.41e-02f  1
Scaling factors are invalid - setting them all to 1.
Scaling factors are invalid - setting them all to 1.
Scaling factors are invalid - setting them all to 1.
 168  5.8970011e+04 3.91e+01 3.01e+10  -1.0 2.08e+03   2.3 1.34e-01 9.37e-02f  1
Scaling factors are invalid - setting them all to 1.
Scaling factors are invalid - setting them all to 1.
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
 169  5.8660464e+04 3.93e+01 2.78e+10  -1.0 2.78e+03   1.8 2.36e-01 1.92e-01f  2
Scaling factors are invalid - setting them all to 1.
Scaling factors are invalid - setting them all to 1.
Scaling factors are invalid - setting them all to 1.
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 170  5.8414402e+04 3.93e+01 2.47e+10  -1.0 1.42e+03   2.2 9.77e-01

 242  3.2512529e+03 5.72e+00 2.22e+09  -1.0 5.21e+01   4.3 8.48e-03 2.08e-03h  4
 243  3.2513061e+03 5.70e+00 7.07e+09  -1.0 5.23e+01   4.7 3.66e-02 1.09e-02h  2
 244  3.2518390e+03 6.75e+00 1.61e+12  -1.0 5.17e+01   4.2 3.51e-02 1.11e-01h  1
 245  3.2559191e+03 5.09e+00 1.32e+12  -1.0 4.62e+01   4.6 9.46e-01 9.30e-01h  1
 246  3.2475059e+03 5.28e+00 2.13e+12  -1.0 9.40e+02   4.2 3.77e-02 3.10e-02F  1
 247  3.2475051e+03 5.28e+00 2.13e+12  -1.0 1.30e+03   7.3 1.07e-02 1.97e-04h  1
 248  3.2475051e+03 5.28e+00 2.13e+12  -1.0 9.60e+01   7.7 3.74e-01 1.58e-05h  1
 249r 3.2475051e+03 5.28e+00 1.00e+03   0.7 0.00e+00   7.3 0.00e+00 4.29e-07R  3
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 250r 3.2475136e+03 2.95e+00 1.00e+03   0.7 4.84e+03    -  2.80e-04 1.08e-03f  1
 251  2.2953163e+03 2.32e+00 3.56e+02  -1.0 5.23e+03    -  2.29e-03 2.73e-01f  1
 252  8.1172905e+02 2.02e+00 2.57e+03  -1.0 4.07e+03    -  7.86e-02 9.85e-01f  1
 253  2.6339436e+02 3.26e+00

 335  1.5315684e+02 5.53e-09 9.33e-08  -8.6 5.84e-03    -  1.00e+00 1.00e+00h  1
 336  1.5315684e+02 2.25e-09 1.24e-07  -9.0 3.85e-01    -  1.00e+00 1.00e+00H  1
 337  1.5315684e+02 4.36e-10 3.76e-07  -9.0 5.11e-03    -  1.00e+00 1.00e+00H  1
 338  1.5315684e+02 6.33e-11 3.63e-08  -9.0 2.73e-05    -  1.00e+00 1.00e+00h  1

Number of Iterations....: 338

                                   (scaled)                 (unscaled)
Objective...............:   1.5315684121310838e+02    1.5315684121310838e+02
Dual infeasibility......:   3.6333711451322246e-08    3.6333711451322246e-08
Constraint violation....:   6.3313798648323427e-11    6.3313798648323427e-11
Complementarity.........:   9.0909090909090931e-10    9.0909090909090931e-10
Overall NLP error.......:   5.0954672628847190e-09    3.6333711451322246e-08


Number of objective function evaluations             = 681
Number of objective gradient evaluations             = 335
Number of equality constraint evaluations            = 1660
Number o

In [10]:
print("The SSE at the optimal solution is %0.6f" % obj_value)
print()
print("The values for the parameters are as follows:")
for k,v in parameters.items():
    print(k, "=", v)

The SSE at the optimal solution is 153.156841

The values for the parameters are as follows:
fs.properties.PR_kappa[R125,bmimPF6] = 0.04738878708457129
fs.properties.PR_kappa[R32,bmimPF6] = -0.05102988879852
fs.properties.PR_kappa[bmimPF6,R125] = 1.6929884880363986
fs.properties.PR_kappa[bmimPF6,R32] = 0.28394219938268744


In [11]:
print(large_residuals_set(pest.ef_instance))

ComponentSet([])
